# Model Training
- Author: Myles Dunlap


This notebook is used to train a model using a single configuration file. The steps in this notebook are used in the Python script.

# User Inputs

In [1]:
# Append the path for the custom modules
path_custom_modules = '../'

# Path to the YAML config. file
path_cfg = {'base_dir': '../cfgs',
            'filename': 'train-0.yaml'}

# Import Libraries and Modules

In [2]:
# Libraries
import sys
import os
from pathlib import Path

import argparse
import inspect

# Append Path to Custom Modules
sys.path.append(path_custom_modules)

# Custom Modules
from src.models import llm_multiclass
from src.utils import (RecursiveNamespace,
                       seed_everything,
                       load_cfg,
                       RunIDs)
from src.dataloading.load_data import LoadData
from src.dataloading.stratify import StratifyData
from src.dataloading.preprocess import PreprocessData
from src.dataloading.load_datasets import (TrainDataset,
                                           CustomTextCollator,
                                           )

# Allow HF tokenizer parallelism
os.environ['TOKENIZERS_PARALLELISM'] = 'True'
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [3]:
# Load the configuration file
CFG = load_cfg(base_dir=Path(path_cfg['base_dir']),
               filename=path_cfg['filename'])

# Set random seed on everything
seed_everything(seed=CFG.seed)

In [4]:
# Group ID and ID for each fold tested
run_ids = RunIDs(test_folds=CFG.cv.val_folds,
                 num_folds=CFG.cv.num_folds)
run_ids.generate_run_ids()

# Print the group id and ids for each fold
print(f'Group ID: {run_ids.group_id}')
for fold_num in CFG.cv.val_folds:
    fold_id = getattr(run_ids.folds_id,
                      f'fold{fold_num}').run_id
    entire_id = f'{run_ids.group_id}-{fold_id}' 
    print((f'ID for Testing Fold #{fold_num}: '
           f'{fold_id}\n\tFull/Entire ID: {entire_id}'))

Group ID: f96d4ac2
ID for Testing Fold #1: c35b8d3c
	Full/Entire ID: f96d4ac2-c35b8d3c
ID for Testing Fold #2: 2757f8cb
	Full/Entire ID: f96d4ac2-2757f8cb
ID for Testing Fold #3: 8fceedf3
	Full/Entire ID: f96d4ac2-8fceedf3
ID for Testing Fold #4: eef8284a
	Full/Entire ID: f96d4ac2-eef8284a
ID for Testing Fold #5: 2a0391f9
	Full/Entire ID: f96d4ac2-2a0391f9


In [5]:
# Load Data from Disk
load_data_file = LoadData(base_dir=CFG.paths.data.base_dir)
if CFG.debug:
    data = load_data_file.load(filename=CFG.paths.data.debug_data)
else:
    data = load_data_file.load(filename=CFG.paths.data.data)


In [6]:
# Stratify the Data
data = (StratifyData(technique=CFG.stratify.technique,
                     n_folds=CFG.cv.num_folds,
                     target=CFG.data_info.target)
            .stratify(df=data))
cols = CFG.data_info.source_fields + \
       [CFG.data_info.target, 'fold']
display(data[cols].head(5))
print(f'Distribution of a Product for Each Fold')
print(f'Notice how the quantities are evenly distributed across folds')
display(data.groupby('Product').fold.value_counts()
        .sort_index().head(5))

,Consumer complaint narrative,State,Company response to consumer,Product,fold
0,The Summer of XX/XX/2018 I was denied a mortga...,IL,Closed with explanation,"Credit reporting, credit repair services, or o...",1
1,There are many mistakes appear in my report wi...,VA,Closed with explanation,"Credit reporting, credit repair services, or o...",1
2,There are many mistakes appear in my report wi...,TX,Closed with explanation,"Credit reporting, credit repair services, or o...",1
3,There are many mistakes appear in my report wi...,TX,Closed with explanation,"Credit reporting, credit repair services, or o...",1
4,There are many mistakes appear in my report wi...,CA,Closed with explanation,"Credit reporting, credit repair services, or o...",1


Distribution of a Product for Each Fold
Notice how the quantities are evenly distributed across folds


Product                  fold
Bank account or service  1       2953
                         2       2953
                         3       2954
                         4       2953
                         5       2953
Name: count, dtype: int64

In [7]:
from transformers import AutoTokenizer, DataCollatorWithPadding
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(f'{CFG.model_tokenizer.base_dir}/'
                                          f'{CFG.model_tokenizer.name}')

# Collator
collator = CustomTextCollator(tokenizer=tokenizer,
                              max_sequence_len=CFG.tokenizer.max_length,
                              )


Start looping over folds here

In [8]:
from torch.utils.data import DataLoader
# Train a model for each validation fold
fold_num = CFG.cv.val_folds[0]

# Split Data into Training and Validation
df_train = data.copy()[data.fold != fold_num].reset_index(drop=True)
df_val = data.copy()[data.fold == fold_num].reset_index(drop=True)
print(f'Train Number of Instances: {len(df_train):,}')
print(f'Validation Number of Instances: {len(df_val):,}')

Train Number of Instances: 305,739
Validation Number of Instances: 76,435


# Preprocess Target

Convert the text target into a label.

In [9]:
# Preprocessing Encoders
encoders = {}
for technique in CFG.preprocessing.apply_techniques:
    fields = getattr(CFG.preprocessing, technique).fields
    for col in fields:
        enc = PreprocessData(y=df_train[col].values,
                             technique=technique)
        encoders[col] = {'encoder': enc.encoder,
                         'technique': technique}

## Combine all mix-data type fields

In this [blob post by Chris McCormick](https://mccormickml.com/2021/06/29/combining-categorical-numerical-features-with-bert/) an interesting approach was taken to combine mixed data types which was to convert all categorical and numerical into the text feed into the LLM. This repository will try this technique ingle string that will be processed by the LLM. 

Another approach is only pass the unstructured text into the LLM, take its last layer output and combine with the other mixed data types into a dense layer. This type of approach seems more common and an example can be found on [Google Colab here](https://colab.research.google.com/drive/1F7COnwHqcLDPg_SS-oFgW3c2GPDWnS5Y#scrollTo=BAQFbN-wBpoz).

In [10]:
# Train Dataset and Dataloader
train_dataset = TrainDataset(df=df_train,
                             tok=tokenizer,
                             tok_cfg=CFG.tokenizer,
                             X_cols=CFG.data_info.source_fields,
                             label=CFG.data_info.target,
                             encoder=encoders[CFG.data_info.target]['encoder'])
train_dataloader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              collate_fn=collator,
                              shuffle=True,
                              num_workers=CFG.num_workers,
                              pin_memory=True)

# Validation Dataset and Dataloader
val_dataset = TrainDataset(df=df_val,
                           tok=tokenizer,
                           tok_cfg=CFG.tokenizer,
                           X_cols=CFG.data_info.source_fields,
                           label=CFG.data_info.target,
                           encoder=encoders[CFG.data_info.target]['encoder'])
val_dataloader = DataLoader(val_dataset,
                            batch_size=CFG.batch_size,
                            collate_fn=collator,
                            shuffle=True,
                            num_workers=CFG.num_workers,
                            pin_memory=True)

print(f'# of Training Instances: {len(df_train):,}')
print(f'Batch Size: {CFG.batch_size}')
print(f'{len(df_train):,} / {CFG.batch_size:,} = {len(train_dataloader):,}')
print(f'Train DataLoader # of Iters: {len(train_dataloader):,}')
print(f'Val. DataLoader # of Iters: {len(val_dataloader):,}')

# of Training Instances: 305,739
Batch Size: 4
305,739 / 4 = 76,435
Train DataLoader # of Iters: 76,435
Val. DataLoader # of Iters: 19,109


In [11]:
for i, inputs in enumerate(train_dataloader):
    if i > 10:
        break
    else:
        shape = inputs['input_ids'].shape
    print(f'{i}: {shape}')

0: torch.Size([4, 512])
1: torch.Size([4, 428])
2: torch.Size([4, 334])
3: torch.Size([4, 512])
4: torch.Size([4, 512])
5: torch.Size([4, 512])
6: torch.Size([4, 512])
7: torch.Size([4, 169])
8: torch.Size([4, 445])
9: torch.Size([4, 125])
10: torch.Size([4, 257])


In [ ]:
a = next(iter(train_dataset))
inputs = a['inputs']
labels = a['labels']
collator({'label', labels})

# print(type(inputs))
print(type(labels))

In [ ]:
i = 0
for step, (a, b) in enumerate(train_dataloader):
    print(i)
    i += 1

# Load the Model

In [ ]:
from transformers import AutoConfig, AutoModel, BertForPreTraining, AutoModelForMaskedLM, BertForMaskedLM
model_cfg = AutoConfig.from_pretrained(Path(CFG.model_tokenizer.base_dir) / CFG.model_tokenizer.name)
model = BertForMaskedLM.from_pretrained(Path(CFG.model_tokenizer.base_dir) / CFG.model_tokenizer.name,
                                  config=model_cfg)

In [ ]:
# for i in range(18):
#     a, b = next(iter(train_dataset), i)
#     print(a['input_ids'].shape)
#     print(b.shape)

In [ ]:
a = next(iter(train_dataset))

# # Train a model for each validation fold
# for fold_num in CFG.stratify.val_folds:
#     print(f'Starting Training for Fold {fold_num}')
    
#     # Training Module
    
#     # Inference Module
    
#     print(f'\tFinished Training for Fold {fold_num}')
# print('Training and Validation Completed')
